This is a simple example of SimSwap on processing video with multiple faces. You can change the codes for inference based on our other scripts for image or single face swapping.

Code path: https://github.com/neuralchen/SimSwap

Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

In [14]:
## make sure you are using a runtime with GPU
## you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

Thu Mar 23 10:51:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    53W / 400W |   3363MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation

All file changes made by this notebook are temporary. 
You can try to mount your own google drive to store files if you want.


In [ ]:
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

Cloning into 'SimSwap'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 362 (delta 149), reused 272 (delta 67), pack-reused 0
Receiving objects: 100% (362/362), 101.31 MiB | 32.47 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Already up to date.


In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

     |████████████████████████████████| 4.5MB 10.2MB/s 
     |████████████████████████████████| 12.2MB 51.4MB/s 


In [3]:
import os
os.chdir("SimSwap")
!ls

 AdaptiveAvgPool2d.patch   output
 antelope.zip		   parsing_model
 arcface_model		   pg_modules
 BatchNorm1d.patch	   predict.py
 BatchNorm2d.patch	   PReLU.patch
 checkpoints		   README.md
 checkpoints.zip	   Sequential.patch
 cog.yaml		   Sigmoid.patch
 Conv2d.patch		  'SimSwap colab.ipynb'
 crop_224		   simswaplogo
 data			   test_one_image.py
 DataParallel.patch	   test_video_swapmulti.py
 demo_file		   test_video_swap_multispecific.py
 docs			   test_video_swapsingle.py
 download-weights.sh	   test_video_swapspecific.py
 Dropout.patch		   test_wholeimage_swapmulti.py
 insightface_func	   test_wholeimage_swap_multispecific.py
 LICENSE		   test_wholeimage_swapsingle.py
 Linear.patch		   test_wholeimage_swapspecific.py
 MaxPool2d.patch	   train.ipynb
 models			   train.py
 MultiSpecific.ipynb	   util
 options


In [ ]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

Archive:  ./checkpoints.zip
   creating: ./checkpoints/people/
  inflating: ./checkpoints/people/iter.txt  
  inflating: ./checkpoints/people/latest_net_D1.pth  
  inflating: ./checkpoints/people/latest_net_D2.pth  
  inflating: ./checkpoints/people/latest_net_G.pth  
  inflating: ./checkpoints/people/loss_log.txt  
  inflating: ./checkpoints/people/opt.txt  
   creating: ./checkpoints/people/web/
   creating: ./checkpoints/people/web/images/


In [ ]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

### Now onedrive file can be downloaded in Colab directly!
### If the link blow is not permanent, you can just download it from the 
### open url(can be found at [our repo]/doc/guidance/preparation.md) and copy the assigned download link here.
### many thanks to woctezuma for this very useful help
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/


--2021-06-21 02:14:17--  https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w
Resolving sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)... 13.107.42.12
Connecting to sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)|13.107.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248024513 (237M) [application/zip]
Saving to: ‘antelope.zip’

antelope.zip        100%[===================>] 236.53M  6.16MB/s    in 31s     

2021-06-21 02:14:48 (7.66 MB/s) - ‘antelope.zip’ saved [248024513/248024513]

Archive:  ./antelope.zip
   creating: ./insightface_func/models/antelope/
  inflating: ./insightface_func/models/antelope/glintr100.onnx  
  inflating: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx  


## Inference

In [ ]:
!pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 torchtext==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [8]:
!python test_video_swapsingle.py --crop_size 224 --use_mask --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path ./demo_file/IMG_3369.jpg --video_path ./demo_file/rec.mov --output_path ./output/multi_test_swapsingle.mp4 --temp_path ./temp_results --no_simswaplogo

------------ Options -------------
Arc_path: arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
crop_size: 224
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: None
output_nc: 3
output_path: ./output/multi_test_swapsingle.mp4
phas